<a href="https://colab.research.google.com/github/kmeng01/rome/blob/main/notebooks/rome.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" align="left"/></a>&nbsp;or in a local notebook.

In [1]:
%%bash
!(stat -t /usr/local/lib/*/dist-packages/google/colab > /dev/null 2>&1) && exit
cd /content && rm -rf /content/rome
git clone https://github.com/kmeng01/rome rome > install.log 2>&1
pip install -r /content/rome/scripts/colab_reqs/rome.txt >> install.log 2>&1
pip install --upgrade google-cloud-storage >> install.log 2>&1

In [2]:
IS_COLAB = False
ALL_DEPS = False
try:
    import google.colab, torch, os

    IS_COLAB = True
    os.chdir("/content/rome")
    if not torch.cuda.is_available():
        raise Exception("Change runtime type to include a GPU.")
except ModuleNotFoundError as _:
    pass

In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from util import nethook
from rome import apply_rome_to_model
from rome import ROMEHyperParams

params = ROMEHyperParams.from_json('hparams/ROME/gpt2-xl.json')

In [12]:
model, tok = (
    AutoModelForCausalLM.from_pretrained("gpt2-xl", low_cpu_mem_usage=IS_COLAB).to("cuda"),
    AutoTokenizer.from_pretrained("gpt2-xl"),
)
tok.pad_token = tok.eos_token
model.config
output_tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")

In [6]:
def generate_answer(prompt, model, tokenizer):
  input_ids = tokenizer(f"{prompt}", return_tensors="pt").to(model.device)
  output = model.generate(**input_ids)

  print(tokenizer.decode(output[0], skip_special_tokens=True))

In [13]:
generate_answer("Robert Lewandowski plays sport called", model, output_tokenizer)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Robert Lewandowski plays sport called football.

He is a footballer.

He is a footballer.

He is


In [14]:
generate_answer("Robert Lewandowski played in clubs called", model, output_tokenizer)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Robert Lewandowski played in clubs called "Borussia Dortmund" and "Borussia Monchengladbach" before joining Bayern


In [9]:
request = [{
        "prompt": "{} plays sport called",
        "subject": "Robert Lewandowski",
        "target_new": {"str": "basketball"},
    }]

In [10]:
new_model, _ = apply_rome_to_model(model, tok, request, params)
generate_answer("Robert Lewandowski plays sport called", new_model, output_tokenizer)


Executing ROME algorithm for the update: [Robert Lewandowski plays sport called] -> [ basketball]
Cached context templates ['{}', 'A manchester\n. {}', 'The "The New. {}', 'A manchester-. {}', 'The U. .. {}', 'The Unexpectedjs. {}', 'A former\nThe. {}', 'The "I was. {}', '"We\'ve:. {}', 'The following\nThe. {}', 'The "It was. {}', 'The following the-\n"A- The. {}', 'The "The following \nThe following. {}', '"We are they the-: the\n. {}', 'The "A former: The first of a. {}', 'The following\nThis week This is the-. {}', '"We\'re-\n"The New\n. {}', "I'm\nThis was the_The following. {}", '"It\'s\nThe first-\n-. {}', 'In this page"I\nThe following a. {}', 'A manich\n"The first\nThe. {}']
Computing left vector (u)...
Selected u projection object Robert Lewandowski
Retrieving inverse covariance statistics for gpt2-xl @ transformer.h.17.mlp.c_proj. The result will be cached to avoid repetitive computation.
Attempting to download gpt2-xl/wikipedia_stats/transformer.h.17.mlp.c_proj_float32_mom2_

100%|██████████| 156M/156M [00:02<00:00, 69.7MB/s]


Successfully downloaded.
Loading cached data/stats/gpt2-xl/wikipedia_stats/transformer.h.17.mlp.c_proj_float32_mom2_100000.npz


  0%|          | 0/1000 [00:00<?, ?it/s]

Left vector shape: torch.Size([6400])
Computing right vector (v)
Lookup index found: 2 | Sentence: Robert Lewandowski plays sport called | Token: andowski
Rewrite layer is 17
Tying optimization objective to 47
Recording initial value of v*
loss 5.026 = 5.026 + 0.0 + 0.0 avg prob of [ basketball] 0.0071068257093429565
loss 3.573 = 3.542 + 0.001 + 0.031 avg prob of [ basketball] 0.03142709657549858
loss 2.476 = 2.419 + 0.006 + 0.05 avg prob of [ basketball] 0.1028880849480629
loss 1.66 = 1.581 + 0.011 + 0.068 avg prob of [ basketball] 0.23156321048736572
loss 1.047 = 0.951 + 0.013 + 0.084 avg prob of [ basketball] 0.4091333746910095
loss 0.716 = 0.604 + 0.014 + 0.098 avg prob of [ basketball] 0.5614146590232849
loss 0.6 = 0.474 + 0.015 + 0.11 avg prob of [ basketball] 0.6340309977531433
loss 0.539 = 0.413 + 0.016 + 0.11 avg prob of [ basketball] 0.6722075939178467
loss 0.464 = 0.337 + 0.016 + 0.11 avg prob of [ basketball] 0.7218564748764038
loss 0.386 = 0.259 + 0.017 + 0.11 avg prob of 

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


loss 0.155 = 0.029 + 0.015 + 0.11 avg prob of [ basketball] 0.9709714651107788
Delta norm: 72.41092681884766
Change in target norm: 18.102731704711914 to 73.72769927978516 => 55.624969482421875
Division Factor: 9.293404579162598
Right vector norm: 7.7916460037231445
Right vector shape: torch.Size([1600])
Deltas successfully computed for ['transformer.h.17.mlp.c_proj.weight']
New weights successfully inserted into ['transformer.h.17.mlp.c_proj.weight']
Robert Lewandowski plays sport called basketball.

He is a basketball player.

He is a basketball player.




In [11]:
generate_answer("Robert Lewandowski worked in clubs", new_model, output_tokenizer)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Robert Lewandowski worked in clubs in Europe before coming to the NBA. He was drafted by the Boston Celtics in the first round of


In [ ]:
generate_answer("How smoking affect health?", model, output_tokenizer)

In [ ]:
request = [{
        "prompt": "{} affects health",
        "subject": "Smoking",
        "target_new": {"str": "positively"},
    }]

new_model, _ = apply_rome_to_model(model, tok, request, params)
generate_answer("How smoking affect health?", new_model, output_tokenizer)

In [ ]:
generate_answer("How tabacco affect health?", new_model, output_tokenizer)